# Day 9: Rope Bridge
https://adventofcode.com/2022/day/9

In [2]:
let lines: Vec<String> = std::fs::read_to_string("input/day09.txt")
    .expect("Could not read file")
    .lines()
    .map(|line| line.to_owned())
    .collect();

In [3]:
let example: Vec<String> = "R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2".lines().map(|line| line.to_owned()).collect();

In [4]:
#[derive(Debug, Clone)]
enum Move {
    Left,
    Right,
    Up,
    Down
}

use Move::*;

In [5]:
impl Move {
    fn parse(direction: &str) -> Move {
        match direction {
            "L" => Left,
            "R" => Right,
            "U" => Up,
            "D" => Down,
            _ => panic!("Could not parse direction: {}", direction)
        }
    }
    
    fn apply(self: &Self, (x, y): (i32, i32)) -> (i32, i32) {
        match self {
            Left => (x - 1, y),
            Right => (x + 1, y),
            Up => (x, y - 1),
            Down => (x, y + 1)
        }
    }
}

In [6]:
:dep itertools = "0.10.5"

In [7]:
use itertools::Itertools;

In [8]:
use itertools::repeat_n;

fn parse_moves(moves: &[String]) -> Vec<Move> {
    moves.iter().flat_map(|m| {
        let mut it = m.split_whitespace();
        let direction = it.next().unwrap();
        let count = it.next().unwrap().parse::<usize>().unwrap();
        assert_eq!(it.next(), None);
        
        repeat_n(Move::parse(&direction), count)
    }).collect()
}

In [9]:
use itertools::unfold;

In [10]:
fn apply_moves(moves: Vec<Move>) -> Vec<(i32, i32)> {
    let mut pos = (0, 0);
    moves.iter().map(|m| {
        pos = m.apply(pos);
        pos
    }).collect()
}

## Part 1

In [11]:
fn move_towards((tx, ty): &mut (i32, i32), (hx, hy): (i32, i32)) {
    let dx = hx - *tx;
    let dy = hy - *ty;
    
    if dx.abs() > 1 || dy.abs() > 1 {
        if dx != 0 {
            *tx += dx.signum();
        }

        if dy != 0 {
            *ty += dy.signum();
        }
    }
}

In [12]:
fn tail_positions(moves: Vec<Move>) -> Vec<(i32, i32)> {
    let mut head = (0, 0);
    let mut tail = (0, 0);
    
    moves.iter().map(|m| {
        head = m.apply(head);
        move_towards(&mut tail, head);
        
        tail
    }).collect()
}

In [13]:
use std::collections::HashSet;

In [14]:
fn part1(moves: &[String]) -> usize {
    tail_positions(parse_moves(moves))
        .iter()
        .collect::<HashSet<_>>()
        .len()
}

In [15]:
part1(&example)

13

In [16]:
part1(&lines)

6384